In [5]:
import cv2 as cv
import numpy as np

- weight file: it's the trained model, the core of the algorithm to detect the objects
- cfg file: it's the configuration file, where there are all the settings of the algorithm
- name files: contains the name of the objects that the algorithm can detect


In [6]:
IMAGE = 'human.jpg'
CONFIG = 'darknet/cfg/yolov3.cfg'
CLASSES = 'darknet/data/coco.names'
WEIGHT = 'darknet/weights/yolov3.weights'

In [7]:
import os
print(os.path.exists(CLASSES))
print(os.path.exists(CONFIG))
print(os.path.exists(IMAGE))
print(os.path.exists(WEIGHT))


True
True
True
True


In [ ]:
# Load Yolo
net = cv.dnn.readNet(WEIGHT, CONFIG)
classes = []
with open(CLASSES, "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Loading image
img = cv.imread(IMAGE)
print(img.shape)

img = cv.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# Detecting objects
blob = cv.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

net.setInput(blob)
outs = net.forward(output_layers)

# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
print(indexes)
font = cv.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv.putText(img, label, (x, y + 30), font, 3, color, 3)


cv.imshow("Image", img)
cv.waitKey(0)
cv.destroyAllWindows()

(960, 960, 3)
[[0]]
